In [ ]:
%%capture
!mkdir new_folder
%cd /content/new_folder/

# download all zipped compressed folders

# Master Plan 2019 Subzone Boundary (No Sea)
!wget -O data4.zip https://data.gov.sg/dataset/c754450d-ecbd-4b7d-8dc1-c07ee842c6d1/download
# Master Plan 2019 Planning Area Boundary
!wget -O data5.zip https://data.gov.sg/dataset/40267ab6-7c08-45c4-b777-a3b10e68f1c8/download
# Electoral Boundary 2020
!wget -O data6.zip https://data.gov.sg/dataset/6241ae7f-6dfe-4351-8570-611357d1a90e/download
# unzip all
!unzip data4.zip && unzip data5.zip && unzip data6.zip && unzip electoral-boundary-dataset.kmz

# CDC dataset from YY
!pip install --upgrade --no-cache-dir gdown

!gdown 1hPxde9qZwt297SsBnkDyb9k4YoYJWDz7
!gdown 127OyOlsGJV5sOX0ej3dINhE8c7gn3_sE
!gdown 1tv32HHtH3A1ZKcj76xvXZavShCIZmjYG
!gdown 1ZLTC3j8gtfiYYih5ffDpP0QbECtOTFrY
!gdown 1Q51nrjV-WkdiJDJSOapSkNg4cE6xTBSI

# rename files to make it clearer

# data4
!mv master-plan-2019-subzone-boundary-no-sea-kml.kml URA2019-Subzone.kml   
!mv master-plan-2019-subzone-boundary-no-sea-geojson.geojson URA2019-Subzone.geojson   

# data5
# planning-boundary-area.kml

# data6
!mv 62C4422C0D5147ED8C28FA94627357DB.xsl electoral2020.xsl
!mv doc.kml electoral2020.kml

# data7
# PA_CDC_Boundary_2020.kml

# download 200k addresses/postal codes
!gdown 1AiVKnBjWelL4O7nUCFBRg99Ns3i9kM_n

# download YN RHS files
!gdown 1-8pEFsIQqHQKHWhUyGDIctv6t2NwML9J
!gdown 1-9dwefd81qvej2C0COgA8sy0-oiNgMzd
!gdown 1-C7KFfEnAj_rLnZM2N_ZgL8-8l-ewJFp

# download YN RHS polyclinics
!gdown 1oC79akiCQSuqmkn2Pp6wlQl0Jx2b7QEG
# download 2020 MOH RHS Excel file
!gdown 1-3vpMQgSYBlahvCb6pfeKshGueQmOmsu

# converts kml files to json files
!pip install kml2geojson

!k2g -sf PA_CDC_Boundary.json PA_CDC_Boundary_2020.kml ./
# !k2g -sf URA2019-Plan.json URA2019-Plan.kml ./
!k2g -sf URA2019-Subzone.json URA2019-Subzone.kml ./
!k2g -sf electoral2020.json electoral2020.kml ./

# remove unnecessary files, careful this might remove all other existing files
# please remember to include "!mkdir new_folder" and "cd new_folder"
import os
contents = os.listdir()
for i in contents:
    if (".kml" not in i) and (".csv" not in i ):
        try:
            os.remove(i)
        except:
            pass

In [2]:
import warnings
warnings.filterwarnings('ignore')
!pip install --quiet geopandas
import geopandas as gpd 
import fiona
import requests
import json
import pandas as pd
from datetime import datetime
import os
import glob

from IPython.display import HTML, display
from ipywidgets import IntProgress
import uuid

from prettytable import PrettyTable
import seaborn as sns 
import matplotlib.pyplot as plt
import numpy as np

# mount google drive to store generated csv files
# to-do: modify for DataBricks
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)
print(os.listdir())


'''

Here are the python packages and related version that we are using for Databricks

geopandas 0.12.2, 
fiona 1.9.0, 
json 2.0.9, 
pandas 1.2.4 --> 1.3.5, 
numpy 1.21.6,
prettytable 3.6.0,
matplotlib 3.2.2,
seaborn 0.11.2,
ipywidgets 7.7.1,

'''

# import prettytable
# import matplotlib
# import ipywidgets
# display(
# "geopandas", gpd.__version__,
# "fiona", fiona.__version__,
# "json", json.__version__,
# "pandas", pd.__version__,
# "numpy", np.__version__,
# "prettytable", prettytable.__version__,
# "matplotlib", matplotlib.__version__,
# "seaborn", sns.__version__,
# "ipywidgets", ipywidgets.__version__,
# )


Mounted at /content/gdrive/
['.config', 'gdrive', 'sample_data']


'\n\nHere are the python packages and related version that we are using for Databricks\n\ngeopandas 0.12.2, \nfiona 1.9.0, \njson 2.0.9, \npandas 1.2.4 --> 1.3.5, \nnumpy 1.21.6,\nprettytable 3.6.0,\nmatplotlib 3.2.2,\nseaborn 0.11.2,\nipywidgets 7.7.1,\n\n'

In [ ]:
def digit_extend(POSTAL_CODE):
    if len(str(POSTAL_CODE)) == 5:
      POSTAL_CODE = '0' + str(POSTAL_CODE)
    return str(POSTAL_CODE)

def add_lnglat(
    df, 
    x_name="X_ADDR", 
    y_name="Y_ADDR", 
    col="POSTAL_ADDR"):
    '''
    Converts (X_ADDR, Y_ADDR) to (longitude, latitude) using OneMap API
    Add new columns lng & lat to df
    '''
    list_of_lnglat = []
    list_of_address = []
    # progress bar
    print(f"Connecting OneMap APIs for LAT/LONG coordinates conversion...")
    f = IntProgress(min=0, max=len(df)) # instantiate the bar
    display(f) # display the bar

    for row in df.iloc:
        # update progress bar
        f.value += 1
        hdr = {"User-Agent": "pandas"}
        try: 
            postcode = row[col]
        except:
            try: 
                postcode = row["POSTAL_CODE"]
            except:
                try:
                    postcode = row["SLA_PostalCode"]
                except:
                    pass
        try:
            # when postcode variables exist
            if len(postcode) == 6 :
                postcode = postcode
            else:
                postcode = digit_extend(postcode)
            url = f'https://developers.onemap.sg/commonapi/search?searchVal={postcode}&returnGeom=Y&getAddrDetails=Y&pageNum=1'
            resp = requests.get(url, headers=hdr)
            extract = json.loads(resp.content)
            lng = extract['results'][0]['LONGITUDE']
            lat = extract['results'][0]['LATITUDE']
            HOUSE_BLK_NO = extract['results'][0]['BLK_NO']
            ROAD_NAME = extract['results'][0]['ROAD_NAME']
            BUILDING_NAME = extract['results'][0]['BUILDING']
        except:
            # when postcode variables do not exist
            try:
                # if x/y coordinates exist
                x, y = row[x_name], row[y_name]
                if len(x) > 0 and float(x):
                    url = f"https://developers.onemap.sg/commonapi/convert/3414to4326?X={x}4&Y={y}"
                    resp = requests.get(url, headers=hdr)
                    extract = json.loads(resp.content)
                    lng = extract['longitude']
                    lat = extract['latitude']
                    HOUSE_BLK_NO = extract['results'][0]['BLK_NO']
                    ROAD_NAME = extract['results'][0]['ROAD_NAME']
                    BUILDING_NAME = extract['results'][0]['BUILDING']
                else:
                    print('Please check function input postcode column name, x/y coordinates column names')
                    # return df
            except:
                # if x/y coordinates do not exist
                print('Please check function input postcode column name, x/y coordinates column names')
                # return df
        list_of_lnglat.append([lng, lat])
        list_of_address.append([ BUILDING_NAME,	HOUSE_BLK_NO,	ROAD_NAME])

    # transpose list_of_lnglat, then list[0] is lng, list[1] is lat
    df[["LONG", "LAT"]] = list_of_lnglat
    df[["BUILDING_NAME",	"HOUSE_BLK_NO",	"ROAD_NAME"]] = list_of_address
    return df

def pipeline_with_internet_connection(
    csv_file_path, 
    col='POSTAL_CODE'):
    '''
        Input: original 200k df (with all columns)
        Output: transformed 200k df (only required information)
    '''
    df = pd.read_csv(csv_file_path, on_bad_lines='skip', dtype=str)
    df = df[[col for col in df.columns if "Unnamed" not in col]]

    # digit add postal codes, padding postcodes to 6-digit string object
    try:
        df["POSTAL_ADDR"] = df.apply(lambda row: digit_extend(row[col]), axis=1)
    except:
        # if the input is only-one-column csv file
        try:
            if df.shape[1] == 1:
                df["POSTAL_ADDR"] = df.apply(lambda row: digit_extend(row[df.columns[0]]), axis=1)
                df = df[["POSTAL_ADDR"]]
        except:
            if len(df.shape) == 1:
                df["POSTAL_ADDR"] = df.apply(lambda row: digit_extend(row), axis=1)
                df = df[["POSTAL_ADDR"]]


    # add latitude & longitude columns using OneMap API, for padded POSTAL_ADDR
    if 'LAT' not in df.columns:
        df = df.drop_duplicates(subset=["POSTAL_ADDR"]) 
        df = add_lnglat(df, col='POSTAL_ADDR')
    return df

# ------------------

# to-do: set the path to kml files for Databricks
def kml_to_df(
    file_name, 
    kml_file_path= ''):
    file_name = kml_file_path + file_name
    if ".kml" in file_name:
        try:
            df = gpd.read_file(file_name, driver='KML')
        except:
            fiona.drvsupport.supported_drivers['KML'] = 'rw'
            df = gpd.read_file(file_name, driver='KML')
    elif ".csv" in file_name:
        df = pd.read_csv(file_name, on_bad_lines='skip')
        df['geometry'] = gpd.GeoSeries.from_wkt(df['geometry'])
        df = gpd.GeoDataFrame(df, geometry='geometry')
    return df

def get_attributes_value(
    html_description, 
    attribute_name='SUBZONE_N'):
    df = pd.DataFrame(pd.read_html(html_description)[0])
    return df[df.iloc[:,0]==attribute_name].values[0][1]
#------------------------------------------------------------#   
def get_pip (
    gdf, 
    regions, 
    new_colname=''):
    '''
    Point in Polygon
    Input:
        df - dataframe with geometry POINT
        regions - dataframe with geometry POLYGON/MULTIPOLYGON
    Output:
        df - original dataframe + each row with info on regions
    '''
    r_list = list(regions.Name)
    # create empty dataframe
    df = pd.DataFrame().reindex_like(gdf).dropna()
    # diaplay progress bar
    max_count = len(r_list)
    print(f"Mapping locations against kml file for {max_count} {new_colname} boundary info...")
    try:
        f = IntProgress(min=0, max=max_count) # instantiate the bar
        display(f) # display the bar
    except:
        pass

    for r in r_list:
        # update progress bar 
        try:
            f.value += 1
        except:
            pass
        #get geometry for specific region
        pol = (regions.loc[regions.Name==r])
        pol.reset_index(drop = True, inplace = True)
        #identify those records from gdf that are intersecting with the region polygon
        pip_mask = gdf.within(pol.loc[0, 'geometry'])
        # pip_mask = gdf.within(Polygon(pol.loc[:, 'geometry']))
        gdf.loc[pip_mask, new_colname] = r
    # Attribute N.A. values will be considered during boundary mapping function
    return gdf

def get_attributes_mapping(
    gdf, 
    kml_file_name_list=[], 
    masterplan={}, 
    kml_df_list={}):
    # attributes mapping
    for kml_file_name in kml_file_name_list:
        plan = masterplan[kml_file_name] 

        kml_df = kml_df_list[kml_file_name]
        if type(plan) is dict and 'kml_' in kml_df["Name"].values[0]:  # kml_1, kml_2, ...
            for col, unicode_name in plan.items():
                print("Boundary Rule Extracting: ", kml_file_name)
                # build mapping for each kml file
                plan[col] = {}
                for name in kml_df["Name"]: 
                    description = kml_df[kml_df["Name"]==name]["Description"].values[0] 
                    html_df = pd.DataFrame(pd.read_html(description)[0])   
                    plan[col][name] = html_df[html_df.iloc[:,0] == unicode_name].values[0][1]  #{'kml_1': "K_M_L", 'kml_2': "K_M_L"}}  
                
                # convert kml_1, kml_2, ... -----> mapping -----> standard name / N.A.
                print("Boundary Rule Casting: ", col, unicode_name)
                gdf[col] = gdf[col].map(plan[col]).fillna(gdf[col])
                    
        elif 'kml_' in kml_df["Name"].values[0]:  
            # RHS: convert kml_1, kml_2, ... -----> mapping -----> standard name
            for col in plan:
                print("Boundary Rule Casting: ", kml_file_name, col, unicode_name, )
                gdf.loc[gdf[col].isin(kml_df['Name'].values), col] = kml_file_name[0:-4]
    return gdf

#------------------------------------------------------------#   
def generate_outliers(
    df, 
    col_list=[
          'CDC_DISTRICT_NAME',	
          'ELD_ELECTORAL_DIVISION_NAME',	
          'URA_REGION_NAME',	
          'URA_PLANNING_AREA_NAME',	
          'URA_PLANNING_SUBZONE_NAME', 
          'MOH_RHS_ZONE_NAME']):
    # This cell is to convert empty values into "N.A.", and save as csv
    queries = ''
    for col in col_list:
            queries += col + ' != ' + col + ' or ' + col + ' == "N.A." or '
    df = df.query(queries[:-3])
    return df

#------------------------------------------------------------#   
def assign_na_values(
    df, 
    col_list=[
            'CDC_DISTRICT_NAME',	
            'ELD_ELECTORAL_DIVISION_NAME',	
            'URA_REGION_NAME',	
            'URA_PLANNING_AREA_NAME',	
            'URA_PLANNING_SUBZONE_NAME', 
            'MOH_RHS_ZONE_NAME']):
    df.reset_index(drop = True, inplace = True)
    for col in col_list:
        df.loc[df.query(
                  col + ' != ' + col
                  ).index, col] = "N.A."
    return df
#------------------------------------------------------------#   
def cleaning(
    df, 
    col_list = [               
            'CDC_DISTRICT_NAME',
            'ELD_ELECTORAL_DIVISION_NAME',
            'URA_REGION_NAME',
            'URA_PLANNING_AREA_NAME',
            'URA_PLANNING_SUBZONE_NAME',
            'MOH_RHS_ZONE_NAME']):

    df['RECORD_CREATED_DT'] = datetime.today().strftime('%Y-%m-%d')
    # final dropping of redundant columns
    df = df[['LAT','LONG',
              'BUILDING_NAME',
              'HOUSE_BLK_NO',
              'ROAD_NAME',
              'POSTAL_ADDR',
              *col_list,
              'RECORD_CREATED_DT']]
    return df
#------------------------------------------------------------#   
# Pipeline of connecting OneMap API for postal code / coordinates convertion, and boundary attributes mapping
def pipeline_without_internet_connection(
    csv_file_path, 
    col='POSTAL_CODE',
    col_list = [               
            'CDC_DISTRICT_NAME',
            'ELD_ELECTORAL_DIVISION_NAME',
            'URA_REGION_NAME',
            'URA_PLANNING_AREA_NAME',
            'URA_PLANNING_SUBZONE_NAME',
            'MOH_RHS_ZONE_NAME'],
      kml_file_path='',
      kml_file_name_list=[
            "URA2019-Subzone.kml", 
            "PA_CDC_Boundary_2020.kml", 
            "electoral2020.kml", 
            "Singapore Health Services.kml" , 
            "National University Health System.kml", 
            "National Healthcare Group.kml"],
      masterplan = {
            # Modify if run for certain kml boundaries
            # Electoral District Boundary
            "electoral2020.kml" : ["ELD_ELECTORAL_DIVISION_NAME"],
            # URA Subzone
            "URA2019-Subzone.kml" : {"URA_PLANNING_SUBZONE_NAME": "SUBZONE_N",
                                "URA_PLANNING_AREA_NAME": "PLN_AREA_N",
                                "URA_REGION_NAME": "REGION_N"},
            # URA with Sea
            'planning-boundary-area.kml': {"URA_PLANNING_AREA_NAME": "PLN_AREA_N",
                                "URA_REGION_NAME": "REGION_N"},
            # CDC 2020
            "PA_CDC_Boundary_2020.kml" : {"CDC_DISTRICT_NAME": "CDC_NAME"},
            # RHS Boundaries
            "Singapore Health Services.kml" : ["MOH_RHS_ZONE_NAME"], 
            "National University Health System.kml" : ["MOH_RHS_ZONE_NAME"], 
            "National Healthcare Group.kml" : ["MOH_RHS_ZONE_NAME"] }):


    df = pd.read_csv(csv_file_path, on_bad_lines='skip', dtype=str)

    # add geopandas Point as column
    gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.LONG, df.LAT))

    # make kml_df list
    kml_df_list = {}
    for kml_file_name in kml_file_name_list:
        kml_df_list[kml_file_name] = kml_to_df(kml_file_name, kml_file_path=kml_file_path)

    # mapping to get kml_n or standard name
    for kml_file_name in kml_file_name_list:
        kml_df = kml_df_list[kml_file_name]
        for col in masterplan[kml_file_name]:
            gdf = get_pip(gdf, kml_df, new_colname=col)

    # attributes mapping
    gdf = get_attributes_mapping(gdf, kml_file_name_list, masterplan, kml_df_list)

    # double-check: blank records ---> 'N.A.'
    gdf = assign_na_values(gdf)

    # fill N.A. with planning-boundary-area.kml
    kml_file_name ='planning-boundary-area.kml'
    kml_df = kml_to_df(kml_file_name, kml_file_path=kml_file_path)
    for item in masterplan[kml_file_name]:
        gdf_ura_na = gdf[gdf[item]=="N.A."]
        gdf_ura_na = get_pip(gdf_ura_na, kml_df, new_colname=item)
    
    # mapping for ura na value improvement, and concatenate back
    gdf_ura_na = get_attributes_mapping(gdf_ura_na, kml_file_name_list=[kml_file_name], masterplan=masterplan, kml_df_list={kml_file_name: kml_df})
    frames = [gdf_ura_na, gdf[~gdf.index.isin(gdf_ura_na.index)]]
    gdf  = pd.concat(frames)

    gdf = cleaning(gdf, col_list=col_list)
    return gdf


if __name__ == '__main__':
    # kml_file_path = '/content/new_folder/'
    kml_file_path = '/content/gdrive/MyDrive/HPB GeoProj Data/'
    # csv_file_path = '/content/new_folder/' + '200kpostal_xy.csv'

    csv_file_path = '/content/gdrive/MyDrive/HPB GeoProj Data/one_col.csv'
    new_csv_file_path = os.path.splitext(csv_file_path)[0] + '_generated.csv'

    df = pipeline_with_internet_connection(csv_file_path=csv_file_path)
    df.to_csv(new_csv_file_path, header=True, index=False)

    df = pipeline_without_internet_connection(csv_file_path=new_csv_file_path, kml_file_path=kml_file_path)
    df.to_csv(new_csv_file_path, header=True, index=False)

    display(df)